# Financial Statement Agent: Automated Balance Sheet Validation with Azure and GPT-4

## Introduction

In this notebook, we will build a **Financial Statement Agent** designed to **automate the validation** of financial balance sheets using a combination of **Azure Document Intelligence** and **GPT-4**. This agent aims to detect **discrepancies** between the provided totals and computed sums of individual items within financial statements, ensuring that assets, liabilities, and equity totals are accurately calculated and consistent with accounting principles.

### Key Objectives:
- **Extract financial data** from balance sheet documents using Azure Document Intelligence.
- **Perform precise summation verification** for key balance sheet categories such as assets, liabilities, and equity.
- **Validate cross-footing and footing** to ensure that the sums of individual items and totals match across years.
- **Flag discrepancies** where provided totals do not match computed sums, and investigate potential resolutions.

### Core Technologies:
1. **Azure Document Intelligence**: We use this service to extract structured financial data from PDFs or scanned images, such as balance sheets. The extracted data will be used as the basis for performing summations and validation checks.
2. **GPT-4 (OpenAI)**: GPT-4 assists in processing and analyzing the extracted data by performing detailed **cross-footing** and **footing** operations, detecting discrepancies, and generating diagnostic reports.

### Approach:
1. **Data Extraction**: Using Azure Document Intelligence, we extract relevant sections of balance sheets (such as current assets and liabilities) and convert the unstructured data into structured JSON format for analysis.
2. **Validation and Error Detection**: We apply predefined rules and financial logic to ensure the **sum of individual items matches the provided totals**. This includes verifying each year’s values for accuracy and tagging mismatches as "DISCREPANCY."
   - **Cross-footing**: Ensuring the sum of items within a section equals the section total.
   - **Footing**: Verifying that total sums in each column match the grand total.
3. **Discrepancy Resolution**: We investigate each discrepancy found, and, if a discrepancy is detected, we attempt to identify **potential resolutions** by matching differences with other values in the balance sheet.
4. **Diagnostic Reporting**: The final output is a **detailed diagnostic report** in Markdown format, summarizing discrepancies and referencing their locations in the balance sheet.

### What You Will Learn:
- How to use **Azure's Document Intelligence** to extract and structure data from balance sheets.
- How to implement **cross-footing and footing calculations** to verify the accuracy of balance sheet totals.
- How to generate **automated reports** that flag errors and discrepancies in financial statements.
- How to use **GPT-4** for intelligent summation verification and to walk through each validation step thoroughly.

### Prerequisites:
- Basic understanding of **financial statement structure** and **accounting principles**.
- Familiarity with **Python** and **API-based integrations**.
- Access to Azure Cognitive Services for using the **Document Intelligence API**.

Let’s begin by extracting and analyzing the assets section of the balance sheet, using Azure Document Intelligence to retrieve the data, followed by GPT-4 to perform summation and validation checks.


In [ ]:
# import libraries
import os
import json
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
from openai import OpenAI

client = OpenAI()

# set `<your-endpoint>` and `<your-key>` variables with the values from the Azure portal
endpoint = "https://aiballstate.cognitiveservices.azure.com/"
key = "56e7553dff434d8693edaaf6e8962214"

## Ignore these for now :)
These are some utility functions needed for later functions. For the purposes of this workshop, we will treat these as a blackbox.

In [ ]:
def parse_value(value):
    # Remove any non-numeric characters like $, commas, etc., and handle negative numbers in parentheses
    if not value:  # Handle empty strings
        return 0.0
    
    value = value.strip()

    value = value.replace('$', '').replace(',', '')
    
    if value.startswith('(') and value.endswith(')'):
        value = '-' + value[1:-1]
    try:
        return float(value)
    except ValueError:
        return 0.0
    
def convert_dict_to_markdown(data):
    markdown_output = ""

    # Add headers
    for line in data["header"]:
        markdown_output += f"{line}\n"
    markdown_output += "\n"

    # Add table
    table = data["table"]
    if table:
        # Ensure all rows have the same number of columns as the header
        num_columns = max(len(row) for row in table)
        for i in range(len(table)):
            table[i] = table[i] + [""] * (num_columns - len(table[i]))

        # Calculate the maximum width of each column
        col_widths = [max(len(str(item)) for item in col) for col in zip(*table)]

        # Function to pad cells
        def pad_cell(cell, width):
            return str(cell).ljust(width)

        # Table header
        header_row = table[0]
        markdown_output += "| " + " | ".join(pad_cell(cell, col_widths[i]) for i, cell in enumerate(header_row)) + " |\n"
        markdown_output += "| " + " | ".join("-" * col_widths[i] for i in range(len(header_row))) + " |\n"

        # Table rows
        for row in table[1:]:
            padded_row = row + [""] * (len(header_row) - len(row))
            markdown_output += "| " + " | ".join(pad_cell(cell, col_widths[i]) for i, cell in enumerate(padded_row)) + " |\n"

    return markdown_output

def extract_content(result, header):
    tbl = ""
    lower_header = [h.lower() for h in header]  
    for res in result:
        if any(h in element.lower() for element in res["header"] for h in lower_header):
            tbl += convert_dict_to_markdown(res)
    return tbl

def validate_bs_totals(balance_sheet, type=None):
    try:
        balance_sheet = balance_sheet.replace('```\n', '').replace('```', '').replace('json\n','')
        json_data = json.loads(balance_sheet)
    except json.JSONDecodeError:
        return "", ""
    validation_results = {}
    res = ""
    stmts = ""
    if type == "liabilities":
        json_data = {key: value for key, value in json_data.items()
             if not any(
                 sub_dict.get("total_label", "").lower() in ["total liabilities", "total liabilities and equity", 
                                                             "total liabilities and members' equity",
                                                             "total liabilities and stockholders' equity"]
                 for sub_dict in value.values()
             )}

    for section, data in json_data.items():
        for year, details in data.items():
            if 'items' not in details or not details['items']:
                continue
            if 'total' not in details or details['total'] is None or details['total'] == "":
                continue
            items_sum = sum(parse_value(v) for v in details['items'].values())
            total_value = parse_value(details['total'])

            validation_results[f"{section} ({year})"] = (items_sum == total_value)
            stmts += f'\n\nSection: {section} ({year})'
            stmts += f'\nSumming up values: {details}'
            stmts += f'\nComputed Sum: {items_sum}'
            stmts += f'\nProvided Sum: {total_value}'
            if not validation_results[f"{section} ({year})"]:
                res += f"\nDISCREPANCY,Balance Sheet,{section} ({year}),Computed Sum: {items_sum} vs. Provided Total: {total_value}. Difference: {total_value - items_sum}"

    return res, stmts



## MSFT Form 10-K 2021 

https://www.sec.gov/Archives/edgar/data/789019/000156459021039151/msft-10k_20210630.htm

In [ ]:
fs_url = "https://raw.githubusercontent.com/alexanderjwhite/2024-AI-Ball-State/main/msft.pdf"

In [ ]:
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key),api_version="2024-07-31-preview"
)

In [ ]:
poller = document_intelligence_client.begin_analyze_document(
    "prebuilt-layout", AnalyzeDocumentRequest(url_source=fs_url)
)

In [ ]:
result = poller.result()

Let's extract only the headers and tables

In [ ]:
tables_with_headers = []

# Extract header and table data from each page
for page in result.pages:
    page_number = page.page_number
    header_lines = []

    # Extract header (first 3 lines of text)
    for line in page.lines[:3]:
        header_lines.append(line.content)

    # Extract table data
    for table in result.tables:
        # Only process tables that belong to the current page
        if (
            table.bounding_regions
            and table.bounding_regions[0].page_number == page_number
        ):
            table_data = []
            for cell in table.cells:
                row_index = cell.row_index
                col_index = cell.column_index
                text = cell.content

                # Ensure the table has enough rows
                while len(table_data) <= row_index:
                    table_data.append([])
                # Ensure the row has enough columns
                while len(table_data[row_index]) <= col_index:
                    table_data[row_index].append("")
                table_data[row_index][col_index] = text

            tables_with_headers.append(
                {"header": header_lines, "table": table_data}
            )

print("DIS prebuilt-layout: Analysis complete.")
result = tables_with_headers

outputs = {
    "combined":"",
    "tables":""
    }


In [ ]:
outputs

In [ ]:
# Define the necessary variables
output_report_path = ""  # Default to current file location
company_name = "MSFT"  # Replace with the actual company name

# Create the report file paths
output_report_path_files = [
    "Balance Sheet Assets Diagnostics.md", 
    "Balance Sheet Liabilities Diagnostics.md", 
    "Balance Sheet Diagnostics.md", 
    "Balance Sheet Validations.md"
]

output_report1, output_report2, output_report3, valid_report = [
    output_report_path + company_name + " - " + file for file in output_report_path_files
]

print(output_report1, output_report2, output_report3, valid_report)


In [ ]:
# Extract balance sheet from the result
balance_sheet = extract_content(result, ["BALANCE SHEETS", "BALANCE SHEET", "BALENCE SHEET"])


In [ ]:
balance_sheet

# Analyze Balance Sheet Assets

In [ ]:
prompt_1 = f"""
From the following tables extracted from the balance sheet, perform the below summations and validations accurately for latest year and previous year. If there is data for one year only, perform the below summations and validations accurately for one year only. 
Be precise, thorough and walk through each calculation and validation step-by-step with as much detail as possible.

1. **Summation Verification:**
- **Balance Sheet:**
    - Verify the sum of all current assets by summing up individual line items. Do not use pre calculated totals in the table for summation.
    - Verify the sum of Property and Equipment by summing up individual line items. Do not use pre calculated totals in the table for summation.
    - Verify the sum of all other assets by summing up individual line items. Do not use pre calculated totals in the table for summation.
    - Verify the sum of total assets by summing up individual line items. Do not use pre calculated totals in the table for summation.
    For each summation above:
    - Cross-foot (verify that the sum of individual line items equals the total provided in the row or section). Pay extra attention to these sums and double-check them. Tag incorrect sums as "DISCREPANCY".
    - Foot (verify that the sum of individual line items in each column total equals the grand total). Pay extra attention to Total sums in the column and double-check the summation. Tag incorrect sums as "DISCREPANCY".
2. When performing calculations, include the computed sum and compare it with the provided Total from the balance sheet. Pay extra attention and double-check the numbers exactly match. If they match, confirm the match. If they do not match, highlight the discrepancy and provide the difference.
3. Review Discrepancies:
    3.1 List any discrepancies found that occur in all the available years.
    3.2 For each discrepancy, identify the Difference between Computed Sum and Provided Total
    3.3 Check if the Difference value is > 1 and is exactly equal to any other item in the Balance sheet. Double check to ensure the values are exactly equal, not close in value. Tag these matches as POTENTIAL RESOLUTION.
    3.4 Re-do your footing and cross-footing calculations for that particular discrepancy item by including knowledge from POTENTIAL RESOLUTIONS. 
    3.5 Compare the new Computed sum to the Provided total for that particular discrepancy item. Pay extra attention and double-check the numbers exactly match. 
    3.6 Note: If the numbers are exactly equal and match, confirm the match. Rounding difference of $1 is not acceptable and must be tagged as DISCREPANCY. If the numbers do not match, tag as DISCREPANCY and provide the difference.
4. **Create a Diagnostic Report:**
- For each discrepancy found, list the discrepancy along with reference to the specific location within the financial statements.
    DISCREPANCY: The computed sum ($19,595,808) does not match the provided total ($19,595,807). Difference: $1

Tables:
{balance_sheet}

Ensure to provide the value exactly as it appears, including any commas or periods.
"""

In [ ]:
bs_assets = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a meticulous financial accounting assistant. Help me with proofing my financial statements."},
        {
            "role": "user",
            "content": prompt_1
        }
    ]
)

In [ ]:
print(bs_assets.choices[0].message.content)

In [ ]:
prompt_2 = """
    From the following tables extracted from the BALANCE SHEET, perform the below actions accurately for current year and previous year. If there is data for one year only, perform the below summations and validations accurately for one year only. 
    Be precise, thorough and walk through each calculation and validation step-by-step with as much detail as possible.
    Ensure to provide the value exactly as it appears, including any commas or periods.

    Identify and extract all the numerical values from the ASSETS section for each year as shown in the provided financial statement snippet. List the numbers in the order they appear for each year.
    1. Identify each section in the balance sheet. A section begins with a header (e.g., "CURRENT ASSETS", "PROPERTY AND EQUIPMENT") and ends before the next section header.
    2. Extract all line items within each section along with their corresponding values for both the current year (2023) and the previous year (2022).

    3. For each subtotal or total labeled in the section (e.g., "Total, at Cost", "Total", "Total Current Assets", "Total Property and Equipment", etc.), create a separate dictionary. The number of dictionaries should be equal to the number of totals (`n`) in the section.
    4. Ensure each dictionary contains:
        - The items leading up to the total.
        - The subtotal or total itself with its label.
    5. If there is any "Less" item (e.g., "Less: Accumulated Depreciation"), treat it as a negative value and ensure the subtraction is reflected in the final total within the corresponding dictionary. Include the following in that dictionary:
        - The total before the "Less" item.
        - The "Less" item itself as a negative value.
        - The final total after applying the "Less" item.          
    Format:
    Extract the numbers for each category ONLY in ASSETS section for both years and list them sequentially. Include a separate key for the totals and their respective labels.

    Example Output Format:
    {
        "PROPERTY AND EQUIPMENT (1st Total)": {
            "2023": {
                "items": {
                    "Land": 510741,
                    "Building and Improvements": 5681397,
                    "Machinery and Equipment": 4167354,
                    "Laboratory and Equipment": 61307,
                    "Office Furniture and Equipment": 1720796,
                    "Kitchen Equipment": 177961
                },
                "total_label": "Total, at Cost",
                "total": 12319956
            },
            "2022": ...
        },
        "PROPERTY AND EQUIPMENT (2nd Total)": {
            "2023": {
                "items": {
                    "Total, at Cost": "1,000",
                    "Less: Accumulated Depreciation": "5,000"
                },
                "total_label": "Total",
                "total": "-4,000"
            },
            "2022": ...
        },
        "PROPERTY AND EQUIPMENT (3rd Total)": {
            "2023": {
                "items": {
                    "Total": "-4,000",
                    "Construction in Process": "-1,000"
                },
                "total_label": "Total Property and Equipment",
                "total": "-5,000"
            },
            "2022": ...
        }
    }

    Tables: """ + balance_sheet + """
    Ensure to provide the value exactly as it appears, including any commas, signs or periods. 
    Only respond with the json output.
"""

In [ ]:
tmp_asset = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a meticulous financial accounting assistant. Help me with proofing my financial statements."},
        {
            "role": "user",
            "content": prompt_2
        }
    ]
)

In [ ]:
print(tmp_asset.choices[0].message.content)

# Analyze Balance Sheet Liabilities

In [ ]:
prompt_3 = f"""
        From the following tables extracted from the balance sheet, perform the below summations and validations accurately for latest year and previous year. If there is data for one year only, perform the below summations and validations accurately for one year only. 
        Be precise, thorough and walk through each calculation and validation step-by-step with as much detail as possible.

        1. **Summation Verification:**
        - **Balance Sheet:**
            - Verify the sum of all current liabilities by summing up individual line items. Do not use pre calculated totals in the table for summation.
            - Verify the sum of all long term liabilities by summing up individual line items. Do not use pre calculated totals in the table for summation.
            - Verify the sum of total liabilities by summing up individual line items. Do not use pre calculated totals in the table for summation.
            - Verify the sum of all shareholders' equity by summing up individual line items. 
            For each summation above:
            - Cross-foot (verify that the sum of individual line items equals the total provided in the row or section). Pay extra attention to these sums and double-check them. Tag incorrect sums as "DISCREPANCY".
            - Foot (verify that the sum of individual line items in each column total equals the grand total). Pay extra attention to Total sums in the column and double-check the summation. Tag incorrect sums as "DISCREPANCY".
        2. When performing calculations, include the computed sum and compare it with the provided Total from the balance sheet if present. Pay extra attention and double-check the numbers exactly match. If they match, confirm the match. If they do not match, highlight the discrepancy and provide the difference.
        3. Review Discrepancies:
            3.1 List any discrepancies found that occur in all the available years.
            3.2 For each discrepancy, identify the Difference between Computed Sum and Provided Total
            3.3 Check if the Difference value is > 1 and is exactly equal to any other item in the Balance sheet. Double check to ensure the values are exactly equal, not close in value. Tag these matches as POTENTIAL RESOLUTION.
            3.4 Re-do your footing and cross-footing calculations for that particular discrepancy item by including knowledge from POTENTIAL RESOLUTIONS. 
            3.5 Compare the new Computed sum to the Provided total for that particular discrepancy item. Pay extra attention and double-check the numbers exactly match. 
            3.6 Note: If the numbers are exactly equal and match, confirm the match. Rounding difference of $1 is not acceptable and must be tagged as DISCREPANCY. If the numbers do not match, tag as DISCREPANCY and provide the difference.
        4. **Create a Diagnostic Report:**
        - For each discrepancy found, list the discrepancy along with reference to the specific location within the financial statements.
            DISCREPANCY: The computed sum ($19,595,808) does not match the provided total ($19,595,807). Difference: $1

        Tables:
        {balance_sheet}

        Ensure to provide the value exactly as it appears, including any commas or periods.
    """

In [ ]:
bs_liabilities = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a meticulous financial accounting assistant. Help me with proofing my financial statements."},
        {
            "role": "user",
            "content": prompt_3
        }
    ]
)

In [ ]:
print(bs_liabilities.choices[0].message.content)

In [ ]:
prompt_4 = """
            From the following tables extracted from the BALANCE SHEET, perform the below actions accurately for the current year and the previous year. If data is available for only one year, perform the summations and validations accurately for that year alone. 
            Be precise, thorough, and provide a detailed step-by-step breakdown of each calculation and validation.
            Ensure to preserve the exact format of the values, including any commas, signs, or periods.

            Identify and extract all numerical values from the LIABILITIES AND STOCKHOLDERS’ EQUITY" sections for each year as shown in the provided financial statement snippet. List the numbers in the order they appear for each year.
            1. Identify each section within the liabilities and equity sections of the balance sheet. A section begins with a header (e.g., "CURRENT LIABILITIES", "LONG-TERM LIABILITIES", "STOCKHOLDERS' EQUITY") and ends before the next section header.
            2. Extract all line items within each section along with their corresponding values for both the current year (2023) and the previous year (2022).
            3. Identify the total for each section, typically labeled at the end of the section (e.g., "Total Current Liabilities", "Total Stockholders' Equity").
            Format:
            Extract the numbers for each category ONLY in the LIABILITIES AND STOCKHOLDERS’ EQUITY" sections for both years and list them sequentially. Include a separate key for the totals and their respective labels.

            Example Output Format:
            {
                "CURRENT LIABILITIES": {
                    "2023": {
                        "items": {
                            "Current Portion of Long-Term Debt": "8,750,000",
                            "Accounts Payable": "147,945,304",
                            "Accrued Expenses": "9,108,008",
                            "Current Portion of Deferred Revenues": "16,080,510",
                            "Current Lease Liability - Operating": "290,984"
                        },
                        "total_label": "Total Current Liabilities",
                        "total": "182,174,806"
                    },
                    "2022": ...
                },
                "STOCKHOLDERS' EQUITY": {
                    "2023": {
                        "items": {
                            "Class A Common Stock, Voting": "40,000",
                            "Class B Common Stock, Nonvoting": "166,950",
                            "Additional Paid-In Capital": "1,658,282",
                            "Retained Earnings": "195,863,150",
                            "Accumulated Other Comprehensive Gain - Cumulative Unrealized Gains on Interest Rate Swap Contracts": "4,255,807"
                        },
                        "total_label": "Subtotal",
                        "total": "201,984,189"
                    },
                    "2022": ...
                },
                "STOCKHOLDERS' EQUITY (After Less)": {
                    "2023": {
                        "items": {
                            "Subtotal": "201,984,189",
                            "Less: Cost of Shares of Common Stock Held in Treasury": "-1,335,454"
                        },
                        "total_label": "Total Stockholders' Equity",
                        "total": "200,648,735"
                    },
                    "2022": ...
                }
            }

            Tables: """ + balance_sheet + """
            Exclude items from the JSON output if total_label is "Total Liabilities" or "Total Liabilities and Equity".
            Ensure to provide the value exactly as it appears, including any commas, signs, or periods. 
            Only respond with the JSON output.
        """

In [ ]:
tmp_liabilities = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a meticulous financial accounting assistant. Help me with proofing my financial statements."},
        {
            "role": "user",
            "content": prompt_4
        }
    ]
)

In [ ]:
print(tmp_liabilities.choices[0].message.content)

In [ ]:
tmp_liabilities_ci, stmts = validate_bs_totals(tmp_liabilities.choices[0].message.content, type='liabilities')

In [ ]:
print(stmts)

In [ ]:
prompt_5 = f"""
    From the following tables extracted from the balance sheet, perform the below summations and validations accurately for latest year and previous year. If there is data for one year only, perform the below summations and validations accurately for one year only. 
    Be precise, thorough and walk through each calculation and validation step-by-step with as much detail as possible.

    1. For each year present, ensure total assets equal the sum of total liabilities and shareholders' equity by summing up individual line items in total assets and total liabilities. 
        - Calculate Total Liabilities and Stockholders' Equity: Add the total current liabilities and total stockholders' equity (or deficit). Note that a stockholders' deficit is a negative number.
        - Compare with Total Assets: Ensure that the calculated sum of total liabilities and stockholders' equity matches the reported total assets.
        - Double Check the Calculation: Clearly outline the steps and verify that the final comparison is correct.
    3. When performing calculations, include the computed sum and compare it with the provided Total from the balance sheet. Pay extra attention and double-check the numbers exactly match. If they match, confirm the match. If they do not match, highlight the discrepancy and provide the difference.
    4. **Create a Diagnostic Report:**
    - For each discrepancy found, list the discrepancy along with reference to the specific location within the financial statements.
        DISCREPANCY: The computed sum ($19,595,808) does not match the provided total ($19,595,807). Difference: $1

    Tables:
    {balance_sheet}

    Ensure to provide the value exactly as it appears, including any commas or periods.
"""

In [ ]:
bs_balance = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a meticulous financial accounting assistant. Help me with proofing my financial statements."},
        {
            "role": "user",
            "content": prompt_5
        }
    ]
)

In [ ]:
print(bs_balance.choices[0].message.content)

# Diagnostic Reports

## Assets

In [ ]:
prompt_6 = f"""
        Extract and print the final ### Diagnostic Report from the following summarized markdown document of a company's financial statements, specifically the balance sheet, statement of income, statement of equity and statement of cash flows.
        1. Only extract the LAST diagnostic report present in the document.
            1.1 If no discrepancies, print empty diagnostic report.
            1.2 If POTENTIAL RESOLUTIONS are present, print these records without printing the initial DISCREPANCY. 
            1.3 For discrepancies without POTENTIAL RESOLUTIONS, print DISCREPANCY. 
        2. Exclude diagnostic reports that have no discrepancies listed.
        3. Only respond with the combined diagnostic report as a csv with columns Error Type,Face Statement,Item (Year),Result. 

        File:
        {bs_assets.choices[0].message.content}

        Example Output:
        Error Type,Type,Face Statement,Item (Year),Result
        DISCREPANCY,Footing,Balance Sheet,Accounts Receivable (2022),Computed Sum: $13,363,982 vs. Provided Total: $13,363,981. Difference: $1
        DISCREPANCY,Cross Footing,Balance Sheet,Inventories (2023),Computed Sum: $10,000,000 vs. Provided Total: $11,000,000. Difference: $1,000,000
        POTENTIAL RESOLUTION,Footing,Balance Sheet,Other Assets (2023),Computed Sum: $1,000 vs. Provided Total: $2,000. Difference: $1,000,000. The difference matches the value of Property and equipment, net 1000.

        If no discrepancies are found, print empty diagnostic report.
        Example Output:
        Error Type,Type,Face Statement,Item (Year),Result
    """

In [ ]:
bs_asset_dr = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a meticulous financial accounting assistant. Help me with proofing my financial statements."},
        {
            "role": "user",
            "content": prompt_6
        }
    ]
)

In [ ]:
print(bs_asset_dr.choices[0].message.content)

In [ ]:
prompt_7 = f"""
        Extract and print the final ### Diagnostic Report from the following summarized markdown document of a company's financial statements, specifically the balance sheet, statement of income, statement of equity and statement of cash flows.
        1. Only extract the LAST diagnostic report present in the document.
            1.1 If no discrepancies, print empty diagnostic report.
            1.2 If POTENTIAL RESOLUTIONS are present, print these records without printing the initial DISCREPANCY. 
            1.3 For discrepancies without POTENTIAL RESOLUTIONS, print DISCREPANCY. 
        2. Exclude diagnostic reports that have no discrepancies listed.
        3. Only respond with the combined diagnostic report as a csv with columns Error Type,Face Statement,Item (Year),Result. 

        File:
        {bs_liabilities.choices[0].message.content}

        Example Output:
        Error Type,Type,Face Statement,Item (Year),Result
        DISCREPANCY,Footing,Balance Sheet,Accounts Receivable (2022),Computed Sum: $13,363,982 vs. Provided Total: $13,363,981. Difference: $1
        DISCREPANCY,Cross Footing,Balance Sheet,Inventories (2023),Computed Sum: $10,000,000 vs. Provided Total: $11,000,000. Difference: $1,000,000
        POTENTIAL RESOLUTION,Footing,Balance Sheet,Other Assets (2023),Computed Sum: $1,000 vs. Provided Total: $2,000. Difference: $1,000,000. The difference matches the value of Property and equipment, net 1000.

        If no discrepancies are found, print empty diagnostic report.
        Example Output:
        Error Type,Type,Face Statement,Item (Year),Result
    """

In [ ]:
bs_liab_dr = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a meticulous financial accounting assistant. Help me with proofing my financial statements."},
        {
            "role": "user",
            "content": prompt_7
        }
    ]
)

In [ ]:
print(bs_liab_dr.choices[0].message.content)

In [ ]:
prompt_8 = f"""
        Extract and print the final ### Diagnostic Report from the following summarized markdown document of a company's financial statements, specifically the balance sheet, statement of income, statement of equity and statement of cash flows.
        1. Only extract the LAST diagnostic report present in the document.
            1.1 If no discrepancies, print empty diagnostic report.
            1.2 If POTENTIAL RESOLUTIONS are present, print these records without printing the initial DISCREPANCY. 
            1.3 For discrepancies without POTENTIAL RESOLUTIONS, print DISCREPANCY. 
        2. Exclude diagnostic reports that have no discrepancies listed.
        3. Only respond with the combined diagnostic report as a csv with columns Error Type,Face Statement,Item (Year),Result. 

        File:
        {bs_balance.choices[0].message.content}

        Example Output:
        Error Type,Type,Face Statement,Item (Year),Result
        DISCREPANCY,Footing,Balance Sheet,Accounts Receivable (2022),Computed Sum: $13,363,982 vs. Provided Total: $13,363,981. Difference: $1
        DISCREPANCY,Cross Footing,Balance Sheet,Inventories (2023),Computed Sum: $10,000,000 vs. Provided Total: $11,000,000. Difference: $1,000,000
        POTENTIAL RESOLUTION,Footing,Balance Sheet,Other Assets (2023),Computed Sum: $1,000 vs. Provided Total: $2,000. Difference: $1,000,000. The difference matches the value of Property and equipment, net 1000.

        If no discrepancies are found, print empty diagnostic report.
        Example Output:
        Error Type,Type,Face Statement,Item (Year),Result
    """

In [ ]:
bs_balance_dr = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a meticulous financial accounting assistant. Help me with proofing my financial statements."},
        {
            "role": "user",
            "content": prompt_8
        }
    ]
)

In [ ]:
print(bs_balance_dr.choices[0].message.content)